## Extraction ## 
- This section is to remove needed inforamtion from the PDF(s)

In [1]:
#import packages 
import io #for input and output operations
import requests #to get() the PDFs or url
from PyPDF2 import PdfReader #package for reading info from the PDFs
import pdfplumber #package for reading info from the PDFs
import re #regular expressions allows a user to search for strings
from itertools import chain # used to merge ranges
import pandas as pd
import numpy as np

In [84]:
url = 'https://www.whitehouse.gov/wp-content/uploads/2022/05/BUILDING-A-BETTER-AMERICA-V2.pdf' #website containing PDFs
r = requests.get(url) #retrieve url
f = io.BytesIO(r.content) #manipulate binary data in-memory; this functions the same as with open(file, 'rb') but serves as a temporary buffer

In [85]:
#this box uses pdfplumber - please remeber this code is specific to using th pdfplumber package and to this document https://www.whitehouse.gov/wp-content/uploads/2022/05/BUILDING-A-BETTER-AMERICA-V2.pdf
def extractPDF(f, extractPages, keywords): #function to extract keywords and trailing information  from needed pages
    with pdfplumber.open(f) as pdf:
        pdfData = [] #initliaze storage for entire pdf data 
        
        for i in extractPages: #python is exclusive for ranges so what ever your end is add 1, this range could be changed to user input for ranges i = actual page number + 1 so i=15 is actually page 14 in the document
            #print(i)
            currentPage = pdf.pages[i] #current page i
            pageContent = currentPage.extract_text() #extract text from current page i

            name = pageContent.split('\n')[1]
            #print(name) #capture the name of the program by splitting by newline (this only works for pdfplumber and this document)
            pageData = {'program': name} #initialize data dictionary
        
            for index, key in enumerate(keywords): 

                if index < (len(keywords) - 1): #condition to not go passed the bounds of the array
                #print(key2)
                    nextKey = keywords[index+1] #access the next keyword
                    pattern = '(?<='+key+')(.*)(?='+nextKey+')' #cannot use $ EOF because it will bypass the bounding keyword and give the remaining page text every time -- (.*) is the same as writing [\s\w\d\W]+
                    #print(pattern)
                    values = re.search(pattern, pageContent, re.IGNORECASE | re.DOTALL)
                    keyValue = values.group().strip() #returns the subtring that matches the regular expression and removes leading and trailing whitespace characters
                    #print(keyValue)
                    pageData[key] = keyValue #add new value pair to dictionary

                else:
                    #print(pattern)
                    eofPattern = '(?<='+keywords[-1]+')(.*)(?=$|\\n)' #this does not handle if the end of page keyword is not put in! would have to use all keywords on page for this solution to work #should you exclude "BUILDING A BETTER AMERICA | GUIDEBOOK MAY 2022" here
                    values = re.search(eofPattern, pageContent, re.IGNORECASE | re.DOTALL)
                    keyValue = values.group().strip() #returns the subtring that matches the regular expression and removes leading and trailing whitespace characters
                    #print(keyValue)
                    pageData[key] = keyValue #add new value pair to dictionary

            #print(pageData)
            pdfData.append(pageData) #append page data to previous pdf data
            #print(pdfData)
    return pdfData



In [86]:
extractPages = chain(range(15,56), range(59,64), range(67,92), range(95,98), range(101,116), range(119,137), range(140,150), range(156,178), range(180,206), range(208,225), range(230,265), range(271,340), range(374,378), range(380,383), range(388,399), range(403, 413), range(414, 428), range(429, 431), range(432,439), range(440,441), range(443,447)) #input multiple ranges because not all pages are relevant (leaving page 177 & 178, 205 & 206, 377 & 378 out for now because they span multiple pages)
keywords = ['Federal Agency:', 'Bureau or Account:', 'Funding amount:', 'Period of availability:', 'Funding mechanism:', 'New program:', 'Recipients:', 'Description:', 'Eligible uses:', 'Federal cost share requirement:', 'Statutory location:'] #list of keywords from PDF
            
extracted = extractPDF(f, extractPages, keywords)

## Clean-Up ## 
- This section is to clean up the extracted data

In [104]:
pdfDT = pd.DataFrame(extracted)
pdfDT.rename(columns={'Federal Agency:':'agency', 'Bureau or Account:':'bureau', 'Funding amount:':'funding_amt', 'Funding mechanism:':'funding_type', 'Recipients:':'eligibility', 'Description:':'description', 'Eligible uses:':'eligible_uses'}, inplace=True)
pdfDT

,program,agency,bureau,funding_amt,Period of availability:,funding_type,New program:,eligibility,description,eligible_uses,Federal cost share requirement:,Statutory location:
0,National Highway Performance Program,Department of Transportation,Federal Highway Administration,"$148,000,000,000",4 year,Formula Grant,No. Existing Program with Changed Eligibilities,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,"Highway and bridge projects, generally on the ...","Typically, 80 percent Federal / 20 percent non...",11101; 11105\nBUILDING A BETTER AMERICA | GUID...
1,Surface Transportation Block Grant Program,Department of Transportation,Federal Highway Administration,"$72,000,000,000",4 year,Formula Grant,No. Existing Program with Changed Eligibilities,States (including District of Columbia),The Surface Transportation Block Grant Program...,The Surface Transportation Block Grant Program...,"Typically, 80 percent Federal / 20 percent non...",11101; 11109\nBUILDING A BETTER AMERICA | GUID...
2,Bridge Formula Program,Department of Transportation,Federal Highway Administration,"$26,675,000,000",4 year,Formula Grant,Yes,States (including District of Columbia and Pue...,The Bridge Formula Program will provide funds ...,"Replace, rehabilitate, preserve, protect, and ...","Typically, 80 percent Federal / 20 percent non...","Division J, Title VIII\nBUILDING A BETTER AMER..."
3,Tribal Transportation Facility Bridges (Bridge...,Department of Transportation,Federal Highway Administration,"$825,000,000",4 year,Distributed through the Tribal Transportation ...,No,Tribes,The Bridge Formula Program sets aside 3 percen...,"Funds can be used to plan, design, engineer, o...",100 percent Federal share,"Division J, Title VIII\nBUILDING A BETTER AMER..."
4,Congestion Mitigation and Air Quality Improvem...,Department of Transportation,Federal Highway Administration,"$13,200,000,000",4 year,Formula Grant,No. Existing Program with Changed Eligibilities,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,Transportation projects that reduce congestion...,"Typically, 80 percent Federal / 20 percent non...",11101; 11115\nBUILDING A BETTER AMERICA | GUID...
...,...,...,...,...,...,...,...,...,...,...,...,...
337,Gulf Hypoxia,Environmental Protection Agency,Environmental Protection Agency,"$60,000,000",Available until expended,Grant,Yes,12 State Members of Gulf Hypoxia Task Force (A...,The Gulf of Mexico Hypoxia Task Force includes...,,No non-Federal cost share required,"Division J, Title VI\nNote: This program was c..."
338,Solid Waste Infrastructure for Recycling Infra...,Environmental Protection Agency,Environmental Protection Agency,"$275,000,000",Available until expended,Cooperative Agreements and/or Grants,Yes,"States (including the District of Columbia, a ...",Communities across the country are burdened by...,Provide grants to implement a strategy to impr...,No non-Federal cost share required,70402\nBUILDING A BETTER AMERICA | GUIDEBOOK M...
339,"Reduce, Reuse, Recycling Education and Outreac...",Environmental Protection Agency,Environmental Protection Agency,"$75,000,000",Available until expended,Cooperative Agreements and/or Grants,Yes,"States, a Unit of Local Government, Tribal Gov...",Communities across the country are burdened by...,To award grants focused on improving material ...,No non-Federal cost share required,70402\nBUILDING A BETTER AMERICA | GUIDEBOOK M...
340,Battery Labeling Guidelines,Environmental Protection Agency,Environmental Protection Agency,"$15,000,000","fiscal year 2022, to remain available until Se...",Contracts,Yes,N/A,This investment will promote the safe handling...,Establishes a program to promote battery recyc...,No non-Federal cost share required,"Division J, Title VI\nBUILDING A BETTER AMERIC..."


In [105]:
pdfDT.drop(columns=['Period of availability:','New program:', 'Federal cost share requirement:', 'Statutory location:'], axis=1, inplace=True) #drop unnecessary info

In [106]:

#put columns in correct order (may not matter)
dbColumns = {'category':0, 'category_label':1, 'agencyId':2, 'eligibilityId':3, 'funding_amt_notes':8, 'funding_bill':10, 'next_milestone':14 ,'application':15 , 'link':16, 'pollution':17, 'technical':18}
for column, position in dbColumns.items(): 
    pdfDT.insert(position, column, np.nan)
pdfDT['link'] = url
pdfDT

,category,category_label,agencyId,eligibilityId,program,agency,bureau,funding_amt,funding_amt_notes,funding_type,funding_bill,eligibility,description,eligible_uses,next_milestone,application,link,pollution,technical
0,NaN,NaN,NaN,NaN,National Highway Performance Program,Department of Transportation,Federal Highway Administration,"$148,000,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,"Highway and bridge projects, generally on the ...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
1,NaN,NaN,NaN,NaN,Surface Transportation Block Grant Program,Department of Transportation,Federal Highway Administration,"$72,000,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Surface Transportation Block Grant Program...,The Surface Transportation Block Grant Program...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
2,NaN,NaN,NaN,NaN,Bridge Formula Program,Department of Transportation,Federal Highway Administration,"$26,675,000,000",NaN,Formula Grant,NaN,States (including District of Columbia and Pue...,The Bridge Formula Program will provide funds ...,"Replace, rehabilitate, preserve, protect, and ...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
3,NaN,NaN,NaN,NaN,Tribal Transportation Facility Bridges (Bridge...,Department of Transportation,Federal Highway Administration,"$825,000,000",NaN,Distributed through the Tribal Transportation ...,NaN,Tribes,The Bridge Formula Program sets aside 3 percen...,"Funds can be used to plan, design, engineer, o...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
4,NaN,NaN,NaN,NaN,Congestion Mitigation and Air Quality Improvem...,Department of Transportation,Federal Highway Administration,"$13,200,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,Transportation projects that reduce congestion...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,NaN,NaN,NaN,NaN,Gulf Hypoxia,Environmental Protection Agency,Environmental Protection Agency,"$60,000,000",NaN,Grant,NaN,12 State Members of Gulf Hypoxia Task Force (A...,The Gulf of Mexico Hypoxia Task Force includes...,,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
338,NaN,NaN,NaN,NaN,Solid Waste Infrastructure for Recycling Infra...,Environmental Protection Agency,Environmental Protection Agency,"$275,000,000",NaN,Cooperative Agreements and/or Grants,NaN,"States (including the District of Columbia, a ...",Communities across the country are burdened by...,Provide grants to implement a strategy to impr...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
339,NaN,NaN,NaN,NaN,"Reduce, Reuse, Recycling Education and Outreac...",Environmental Protection Agency,Environmental Protection Agency,"$75,000,000",NaN,Cooperative Agreements and/or Grants,NaN,"States, a Unit of Local Government, Tribal Gov...",Communities across the country are burdened by...,To award grants focused on improving material ...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
340,NaN,NaN,NaN,NaN,Battery Labeling Guidelines,Environmental Protection Agency,Environmental Protection Agency,"$15,000,000",NaN,Contracts,NaN,N/A,This investment will promote the safe handling...,Establishes a program to promote battery recyc...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN


In [107]:
pdfDT = pdfDT.replace('\n',' ', regex=True) #just in case there are new line breaks in the decriptions but may need to clearn further to join disjointed words
pdfDT

,category,category_label,agencyId,eligibilityId,program,agency,bureau,funding_amt,funding_amt_notes,funding_type,funding_bill,eligibility,description,eligible_uses,next_milestone,application,link,pollution,technical
0,NaN,NaN,NaN,NaN,National Highway Performance Program,Department of Transportation,Federal Highway Administration,"$148,000,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,"Highway and bridge projects, generally on the ...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
1,NaN,NaN,NaN,NaN,Surface Transportation Block Grant Program,Department of Transportation,Federal Highway Administration,"$72,000,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Surface Transportation Block Grant Program...,The Surface Transportation Block Grant Program...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
2,NaN,NaN,NaN,NaN,Bridge Formula Program,Department of Transportation,Federal Highway Administration,"$26,675,000,000",NaN,Formula Grant,NaN,States (including District of Columbia and Pue...,The Bridge Formula Program will provide funds ...,"Replace, rehabilitate, preserve, protect, and ...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
3,NaN,NaN,NaN,NaN,Tribal Transportation Facility Bridges (Bridge...,Department of Transportation,Federal Highway Administration,"$825,000,000",NaN,Distributed through the Tribal Transportation ...,NaN,Tribes,The Bridge Formula Program sets aside 3 percen...,"Funds can be used to plan, design, engineer, o...",NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
4,NaN,NaN,NaN,NaN,Congestion Mitigation and Air Quality Improvem...,Department of Transportation,Federal Highway Administration,"$13,200,000,000",NaN,Formula Grant,NaN,States (including District of Columbia),The Bipartisan Infrastructure Law continues th...,Transportation projects that reduce congestion...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,NaN,NaN,NaN,NaN,Gulf Hypoxia,Environmental Protection Agency,Environmental Protection Agency,"$60,000,000",NaN,Grant,NaN,12 State Members of Gulf Hypoxia Task Force (A...,The Gulf of Mexico Hypoxia Task Force includes...,,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
338,NaN,NaN,NaN,NaN,Solid Waste Infrastructure for Recycling Infra...,Environmental Protection Agency,Environmental Protection Agency,"$275,000,000",NaN,Cooperative Agreements and/or Grants,NaN,"States (including the District of Columbia, a ...",Communities across the country are burdened by...,Provide grants to implement a strategy to impr...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
339,NaN,NaN,NaN,NaN,"Reduce, Reuse, Recycling Education and Outreac...",Environmental Protection Agency,Environmental Protection Agency,"$75,000,000",NaN,Cooperative Agreements and/or Grants,NaN,"States, a Unit of Local Government, Tribal Gov...",Communities across the country are burdened by...,To award grants focused on improving material ...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
340,NaN,NaN,NaN,NaN,Battery Labeling Guidelines,Environmental Protection Agency,Environmental Protection Agency,"$15,000,000",NaN,Contracts,NaN,N/A,This investment will promote the safe handling...,Establishes a program to promote battery recyc...,NaN,NaN,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN


In [122]:
pdfDT['agency'].unique() #get unique values
abbr = {'Department of Transportation': 'dot', 'Department of Defense – Army Corps of Engineers' :'dod-ace', 'General Services Administration':'gsa', 'Department of Homeland Security':'dhs', 'Environmental Protection Agency':'epa', 'Department of Energy':'doe',
        'Department of the Interior':'doi', 'Department of Agriculture':'doa', 'Department of Health and Human Services':'dhhs', 'Department of Commerce':'doc', 'Federal Communications Commission':'fcc', 'Denali Commission':'denali', 'Appalachian Regional Commission':'arc',
       'Delta Regional Authority':'dra', 'Northern Border Regional Commission':'nbrc','Southeast Crescent Regional Commission':'scrc'} #create dictionary of agency abbreviates (how to handle subbranch of dod?)

pdfDT['agencyId'] = pdfDT['agency'].map(abbr) #map abbreviations to agency names

pdfDT

    category  category_label agencyId eligibilityId  \
68       NaN             NaN      dot                 

                                              program  \
68  Public Transportation Technical Assistance and...   

                          agency                          bureau funding_amt  \
68  Department of Transportation  Federal Transit Administration    27545852   

    funding_amt_notes                                    funding_type  \
68                NaN  Cooperative Agreement or Interagency Agreement   

    funding_bill                       eligibility  \
68           NaN  National nonprofit organizations   

                                          description  \
68  Provides funding to support workforce developm...   

                                        eligible_uses  next_milestone  \
68  Workforce development, technical assistance an...             NaN   

    application                                               link  pollution  \
68          Na

In [124]:
eligID = {'tribes':'.tribal', 'tribal':'.tribal','states':'.stateGov', 'nonprofits':'.ngo', 'local':'.localGov','community':'.communitygroup','university':'.ed', 'education':'.ed','agricultural':'.agricultural'} #what's the difference between stategov, localgov, and statescountiescities, how to code .statesCountiesCities and .indiv 
#pdfDT['eligibilityId'] = pdfDT['eligibility'].map(eligID)
eligKeys = list(eligID.keys()) # lists out all the keys of the key value pairs

for index, row in pdfDT.iterrows(): #goes through each row of the dataframe
    pattern = r'\b({})\b'.format('|'.join(map(re.escape, eligKeys)))
    # unique matches
    matches = set(re.findall(pattern, row['eligibility'].lower(), re.IGNORECASE))
    values = {eligID[x] for x in matches}
    pdfDT.loc[index,"eligibilityId"] = ', '.join(values)
    
    #print(IDlist)

#print(', '.join(matches)) #PRINTS IT IN LIST FORM
pdfDT
print(pdfDT.loc[pdfDT['program'] == 'Research, Development, Demonstration and Deployment Projects'])

    category  category_label agencyId  \
63       NaN             NaN      dot   

                               eligibilityId  \
63  .communitygroup, .ed, .stateGov, .tribal   

                                              program  \
63  Research, Development, Demonstration and Deplo...   

                          agency                          bureau  \
63  Department of Transportation  Federal Transit Administration   

     funding_amt  funding_amt_notes  \
63  $132,218,677                NaN   

                                         funding_type  funding_bill  \
63  Cooperative Agreement, Contract, Competitive G...           NaN   

                                          eligibility  \
63  States, Counties, Cities / Townships, Special ...   

                                          description  \
63  Provides funding to assist innovative projects...   

                                        eligible_uses  next_milestone  \
63  Research, innovative development, deplo

#### Edge Cases (&Upgrades)... [and general todos] to work on: ####
- More than one page of pdf
- New line break for next word or middle of a word
- Make if/elif/else statement for '.indiv'
- Eligibility is listed as varies, broad eligibility, open
- Agricultural eligibility listed elsewhere 

#### Questions??? ####
- Should I get rid of .stateCitiesCounties and just file those under .localGov

## Scratch ##

In [39]:
#this box uses PyPDF2 - stopped using PyPDF2 because of weird spaces added 

reader = PdfReader(f)
pdftext = [] #initialize holder for all pdfs
#extract text from all needed pages
#for i in range(15,55): #USE len(reader.pages) as your stop when you are ready to do the whole document
pagecontent = ''.join(reader.pages[15].extract_text().split('\n')) #.join to join all the lists created by split() #use i when doing range, use 15 when testing singular page
pagecontent2 = reader.pages[15].extract_text()

#------------Agency----------------#
pattern = '(?<=Statutory Location: )[\s\w\d\W]+(?=$| Federal Agency: )' #do not include the keyword, include any matches with whitespace or alphanumeric characters with more the one instance, include until new line, EOF, or other keywor
#pattern = '(?<=Funding amount: )[\s\w\d\W]+(?=Period of availability: )'
#pattern2 = r'\bFederal Agency: ' #try this out some more
#keywords = ['Federal Agency: ', 'Funding Amount: ', 'Bureau or Account: ']
#pattern3 = rf"{re.escape(keywords[2])}\s*(.*?)(?=\n|{'|'.join(re.escape(k) for k in keywords if k != keywords[2])})"
agency = re.search(pattern, pagecontent, re.IGNORECASE | re.DOTALL)
#print(agency.group())

#----------Funding Amount-----------#
#pattern = '(?<=\$)\d+' #lookbehind a dollar sign
pattern = '\$[\d.,]+' # \$ regular character $ [\d.,] include digits,dot,comma  + one or more
funding = re.search(pattern, pagecontent) #search text for amount matching the funding pattern
funding_amt = funding.group() #retrieves actual text that matches



#pdftext.append(pagecontent)
#pdftext

In [228]:
A = [1,2,3,4]
shiftA = [A[-1]] + A[:-1]

for keyword in keywords:
    (k for k in keywords if k != keyword) #generator expression if range is large or inifite 
    #for i in list: print(i)

print('second loop')
for keyword in keywords:
    list = [k for k in keywords if k != keyword] #list comprehension for multiple iterations or need for speed
    #for i in list: print(i)


for index, key in enumerate(keywords): 
        key2 = [keywords[index+1] if index < (len(keywords) - 1) else keywords[0]]
        print(key2)

second loop
['Bureau or Account: ']
['Funding amount: ']
['Federal Agency: ']


In [51]:
#for ind in pdfDT.index:
   # print(pdfDT['eligibilityId'].loc[ind], pdfDT['eligibility'].loc[ind])
a = ['a', 'b', 'c']
s = "a123b"
#s = "States, local governments, and nonprofits both with and without 501()(3) status"

tmp = {'a':'Aaaa','b':'Bbbb', 'c':'Cccc'}
tmplist = list(tmp.keys())
matches = {tmp[x] for x in s.lower() if x in tmplist}
print(matches)

#t = "States, local governments, and nonprofits both with and without 501(c)(3) status"
#eligID = {'tribes':'.tribal', 'states':'.stateGov', 'nonprofits':'.ngo', 'local':'.localGov','community':'.communitygroup','university':'.ed', 'education':'.ed','agricultural':'.agricultural'} #what's the difference between stategov, localgov, and statescountiescities, how to code .statesCountiesCities and .indiv 
#pdfDT['eligibilityId'] = pdfDT['eligibility'].map(eligID)
#eligKeys = list(eligID.keys()) # lists out all the keys of the key value pairs

#print(eligKeys)
#matches = {eligID[x] for x in t.lower() if x in eligKeys}
#for index, row in pdfDT.iterrows(): #goes through each row of the dataframe
    #matches = {eligID[x] for x in (row['eligibility']).lower() if x in eligKeys} #s is the string or in this case the eligibility description, this shows the matches b/w the eligibility description and the eligiility keys and spits out a list of corresponding ID values, there is definitely a way to do this regex so I don't have to use .lower() but will figure that out later
    #row['eligibilityId'] = ', '.join(matches)  
#print('Set = '+', '.join(matches))

data = {'Name': ['Ankit', 'Amit',
                 'Aishwarya', 'Priyanka'],
        'Age': [21, 19, 20, 18],
        'Stream': ['Math', 'Commerce',
                   'Arts', 'Biology'],
        'Percentage': [88, 92, 95, 70]}
  
# Convert the dictionary into DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age', 
                                 'Stream', 'Percentage'])
#for index, row in df.iterrows():
   #row["New"] = 'Dr. '+ row["Name"]
   #print(row["Name"], row["Age"], row["New"])

{'Aaaa', 'Bbbb'}


In [ ]:
# 'str' is a built-in function, so use another name instead
string = 'A Test test string from which I want to match, multiple substrings for our States and local governments'
#values = ['test', 'test2', 'Multiple', 'ring', 'match', 'local', 'state']
eligID = {'tribes':'.tribal', 'states':'.stateGov', 'nonprofits':'.ngo', 'local':'.localGov'}
values = list(eligID.keys())

pattern = r'\b({})\b'.format('|'.join(map(re.escape, values)))

# unique matches
matches = set(re.findall(pattern, string.lower(), re.IGNORECASE))
IDlist = {eligID[x] for x in matches}
print(IDlist)